In [1]:
import os
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tqdm import tqdm
import random

In [ ]:
BASE_DIR = "transforming-fashion-2025/Dataset/Train/"
IMAGE_DIR = os.path.join(BASE_DIR, "images")
LABEL_DIR = os.path.join(BASE_DIR, "labels")

CLASS_NAMES = {
    0: "sunglass", 1: "hat", 2: "jacket", 3: "shirt", 4: "pants",
    5: "shorts", 6: "skirt", 7: "dress", 8: "bag", 9: "shoe"
}



In [ ]:
def load_annotations(label_path):
    with open(label_path, 'r') as f:
        lines = f.readlines()
    annotations = []
    for line in lines:
        cls, cx, cy, w, h = map(float, line.strip().split())
        annotations.append({
            'class_id': int(cls),
            'center_x': cx,
            'center_y': cy,
            'width': w,
            'height': h
        })
    return annotations

def plot_image_with_boxes(image_path, annotations):
    img = cv2.imread(image_path)
    if img is None:
        return
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w, _ = img.shape
    fig, ax = plt.subplots()
    ax.imshow(img)

    for ann in annotations:
        cx = ann['center_x'] * w
        cy = ann['center_y'] * h
        bw = ann['width'] * w
        bh = ann['height'] * h
        x0 = cx - bw / 2
        y0 = cy - bh / 2
        rect = plt.Rectangle((x0, y0), bw, bh, linewidth=2, edgecolor='lime', facecolor='none')
        ax.add_patch(rect)
        class_name = CLASS_NAMES.get(ann['class_id'], str(ann['class_id']))
        ax.text(x0, y0 - 5, class_name, color='white', fontsize=10,
                bbox=dict(facecolor='black', alpha=0.6, edgecolor='none'))
    ax.axis('off')
    plt.show()

## Alle afbeeldingen en labels samen

In [ ]:
image_files = sorted([f for f in os.listdir(IMAGE_DIR) if f.endswith((".jpg", ".png", ".jpeg"))])


records = []

print("Labels laden...")
for img_file in tqdm(image_files):
    img_path = os.path.join(IMAGE_DIR, img_file)
    label_path = os.path.join(LABEL_DIR, os.path.splitext(img_file)[0] + ".txt")
    if not os.path.exists(label_path):
        continue
    annotations = load_annotations(label_path)
    for ann in annotations:
        ann["image"] = img_file
        records.append(ann)

df = pd.DataFrame(records)
df.head()

## Klasseverdeling

In [ ]:
plt.figure(figsize=(10, 4))
sns.countplot(data=df, x="class_id", palette="Set2")
plt.title("Aantal objecten per klasse")
plt.xticks(ticks=range(10), labels=[CLASS_NAMES[i] for i in range(10)], rotation=45)
plt.xlabel("Klasse")
plt.ylabel("Aantal")
plt.tight_layout()
plt.show()

## Voorbeeld afbeeldingen

In [ ]:
N_VIS = 5  # Aantal afbeeldingen om te visualiseren

print(f"{N_VIS} voorbeeldafbeeldingen met bounding-boxes:")
for fname in random.sample(image_files, k=min(N_VIS, len(image_files))):
    label_path = os.path.join(LABEL_DIR, os.path.splitext(fname)[0] + ".txt")
    anns = load_annotations(label_path)
    plot_image_with_boxes(os.path.join(IMAGE_DIR, fname), anns)

## Co-occurrence matrix (welke items komen samen in één afbeelding)

In [ ]:
# Create a binary matrix: rows = images, columns = classes
pivot_df = df.pivot_table(index="image", columns="class_id", aggfunc="size", fill_value=0)
co_matrix = (pivot_df.T @ pivot_df)

plt.figure(figsize=(10, 8))
sns.heatmap(co_matrix, annot=True, fmt="d", cmap="YlGnBu",
            xticklabels=[CLASS_NAMES[i] for i in range(10)],
            yticklabels=[CLASS_NAMES[i] for i in range(10)])
plt.title("Co-occurrence matrix van objectklassen")
plt.tight_layout()
plt.show()

## Aantal objecten per afbeelding

In [ ]:
object_counts = df.groupby("image").size()

plt.figure(figsize=(8, 4))
sns.histplot(object_counts, bins=20)
plt.title("Aantal objecten per afbeelding")
plt.xlabel("Aantal objecten")
plt.ylabel("Aantal afbeeldingen")
plt.show()

## Overeenkomende items in captioning-dataset en de objectdetectiedatase

- De objectdetectie-dataset bevat 10 klassen:
0 = sunglass
1 = hat
2 = jacket
3 = shirt
4 = pants
5 = shorts
6 = skirt
7 = dress
8 = bag
9 = shoe

- De captioning dataset bevat Engels tekstuele bijschriften van kledingstukken. Veelvoorkomende begrippen in die bijschriften zijn:
“dress”
“jacket”
“skirt”
“shirt”
“pants”
“shorts”
“shoes”
“bag”
“hat”
“sunglasses”

Conclusie: alle 10 klassen van de objectdetectie-dataset komen ook voor als modetermen in de captioning-dataset. Ze zijn dus volledig overlappend en compatibel voor multimodale taken zoals bijschriften genereren per gedetecteerd object.

## Belangrijkste bevindingen van de EDA

- **Klasseverdeling:** Alle 10 objectklassen (zoals 'sunglass', 'hat', 'jacket', etc.) zijn goed vertegenwoordigd in de dataset, maar sommige klassen komen vaker voor dan andere. Vooral 'shirt', 'shoe' en 'jacket' zijn veelvoorkomend.

- **Aantal objecten per afbeelding:** De meeste afbeeldingen bevatten tussen de 2 en 5 objecten, met enkele uitschieters naar boven.

- **Co-occurrence matrix:** Bepaalde kledingstukken komen vaak samen voor in één afbeelding, bijvoorbeeld 'shirt' met 'pants' of 'shoe' met 'dress'. Dit wijst op realistische combinaties van kledingstukken.

- **Voorbeeldafbeeldingen:** De bounding boxes zijn over het algemeen goed geplaatst en de annotaties zijn visueel controleerbaar.

- **Overlap met captioning-dataset:** Alle objectklassen uit de objectdetectie-dataset komen ook als termen voor in de captioning-dataset. Dit maakt de datasets volledig compatibel voor multimodale taken.

# **Deel 1**

# Objectdetectie YOLO


In [ ]:
# Split images into train and validation sets
train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=42)

# Create directories for train and validation sets
train_img_dir = os.path.join(BASE_DIR, "train", "images")
train_label_dir = os.path.join(BASE_DIR, "train", "labels")
val_img_dir = os.path.join(BASE_DIR, "val", "images")
val_label_dir = os.path.join(BASE_DIR, "val", "labels")

os.makedirs(train_img_dir, exist_ok=True)
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_img_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)

# Copy images and labels to the new directories
for img_file in tqdm(train_files, desc="Copying training files"):
    label_file = os.path.splitext(img_file)[0] + ".txt"
    os.rename(os.path.join(IMAGE_DIR, img_file), os.path.join(train_img_dir, img_file))
    if os.path.exists(os.path.join(LABEL_DIR, label_file)):
      os.rename(os.path.join(LABEL_DIR, label_file), os.path.join(train_label_dir, label_file))


for img_file in tqdm(val_files, desc="Copying validation files"):
    label_file = os.path.splitext(img_file)[0] + ".txt"
    os.rename(os.path.join(IMAGE_DIR, img_file), os.path.join(val_img_dir, img_file))
    if os.path.exists(os.path.join(LABEL_DIR, label_file)):
      os.rename(os.path.join(LABEL_DIR, label_file), os.path.join(val_label_dir, label_file))

In [ ]:
# Update YOLO data dictionary with new directory paths
yolo_data_yaml = {
    'train': train_img_dir,
    'val': val_img_dir,
    'nc': 10,
    'names': [CLASS_NAMES[i] for i in range(10)]
}

# Save the updated YAML file
yaml_path = "custom_yolo_data.yaml"
with open(yaml_path, "w") as f:
    yaml.dump(yolo_data_yaml, f)

print(f"Updated {yaml_path} with train: {train_img_dir} and val: {val_img_dir}")

In [ ]:
model = YOLO('yolov8n.pt')

results = model.train(
    data=yaml_path,
    epochs=10,
    imgsz=640,
    project='yolo_train',
    name='custom_clothes'
)

In [ ]:
# Laad het getrainde model
model = YOLO('/content/yolo_train/custom_clothes4/weights/best.pt')

# Evalueer het model op de validatieset
# Gebruik de 'metrics' methode na een 'val' run
results = model.val(data=yaml_path)

print("\nEvaluatie Resultaten:")
print(f"  mAP@0.5: {results.box.map50:.4f}")
print(f"  mAP@0.5:0.95: {results.box.map:.4f}")

### Hoe doet het model voorspellingen?
Het model dat gebruikt wordt is een YOLOv8 objectdetectiemodel. Na het trainen wordt het model geladen met het pad naar het beste gewichtenbestand (`YOLO('/content/yolo_train/custom_clothes4/weights/best.pt')`). Om voorspellingen te doen, geef je een nieuwe afbeelding aan het model. Het model analyseert de afbeelding en geeft voor elk gedetecteerd object een bounding box, een klassenlabel (zoals 'shirt', 'shoe', etc.) en een waarschijnlijkheidsscore terug. Dit gebeurt automatisch voor alle objecten die het model herkent in de afbeelding.


### Hoe worden de afbeeldingen voorbewerkt?
Voor zowel training als voorspellen worden de afbeeldingen automatisch geschaald naar een vaste resolutie van 640x640 pixels (`imgsz=640`). De pixelwaarden worden genormaliseerd zodat het model sneller en stabieler kan leren. Tijdens training worden er ook augmentaties toegepast, zoals draaien, spiegelen en croppen, om het model robuuster te maken tegen variaties in de data. Deze voorbewerkingen worden automatisch uitgevoerd door de YOLO pipeline.


### Welke stappen onderneem je voor feature engineering?
Bij YOLO is handmatige feature engineering nauwelijks nodig. Het model leert zelf welke kenmerken belangrijk zijn voor het herkennen van objecten. Wel worden de bounding boxes genormaliseerd (waarden tussen 0 en 1 ten opzichte van de afbeeldingsgrootte) en worden augmentaties toegepast om de dataset te verrijken. Dit helpt het model om beter te generaliseren naar nieuwe, ongeziene situaties.


### Keuze van hyperparameters, optimizer en aantal epochs
- **Epochs:** Er is gekozen voor 10 epochs (`epochs=10`). Dit is een praktisch startpunt om snel te zien of het model leert. Meer epochs kunnen de prestaties verbeteren, maar kosten meer rekentijd.
- **Optimizer:** YOLOv8 gebruikt standaard de AdamW-optimizer. Deze optimizer is geschikt voor vision-taken omdat hij adaptief leert en gewichtsregularisatie toepast, wat overfitting helpt voorkomen.
- **imgsz:** De afbeeldingsgrootte is 640 pixels. Dit is een goede balans tussen nauwkeurigheid en rekentijd; grotere afbeeldingen kunnen nauwkeuriger zijn, maar zijn ook zwaarder om te trainen.
- **Andere parameters** zoals batch size zijn standaard gelaten, omdat deze al goed zijn afgestemd voor YOLO.


### Beschrijving van de lossfunctie
De lossfunctie van YOLO bestaat uit drie hoofdonderdelen:
- **Bounding box regression loss:** Meet hoe goed de voorspelde bounding boxes overeenkomen met de echte (meestal met CIoU- of GIoU-loss).
- **Objectness loss:** Meet of het model correct voorspelt of er een object aanwezig is in een bepaald gebied van de afbeelding.
- **Classificatie loss:** Meet of het model de juiste klasse voorspelt voor elk gedetecteerd object.

De totale loss is een gewogen som van deze drie onderdelen. Tijdens het trainen probeert het model deze totale loss te minimaliseren, zodat het steeds betere en nauwkeurigere voorspellingen doet.

# **Deel 2&3**

## Importeren van benodigde bibliotheken

In deze stap worden alle benodigde bibliotheken ingeladen die essentieel zijn voor het bouwen, trainen en evalueren van een image captioning-model.

We gebruiken:
- huggingface_hub en getpass voor veilige toegang tot modellen en datasets via Hugging Face
- datasets om kant-en-klare image-text datasets zoals H&M Captions in te laden
- torchvision en PIL voor het transformeren en visualiseren van afbeeldingsdata
- transformers om het ViT-GPT2 encoder-decoder model te gebruiken
- torch en DataLoader om met batches te trainen en het model efficiënt te verwerken
- evaluate voor de berekening van evaluatiemetrieken zoals BLEU-score
- tqdm voor een overzichtelijke voortgangsbalk tijdens training
- matplotlib en random voor het tonen en willekeurig selecteren van afbeeldingen

In [ ]:
# Imports
from huggingface_hub import login
import getpass
from datasets import load_dataset
from torchvision import transforms
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
from transformers.models.blip import BlipProcessor, BlipForConditionalGeneration
from transformers.training_args import TrainingArguments
from transformers.trainer import Trainer
from torch.utils.data import DataLoader
import torch
import evaluate
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import random
import torch

## Inloggen bij Hugging Face

Om toegang te krijgen tot bepaalde datasets en modellen op het Hugging Face platform is authenticatie vereist. We loggen daarom in met een persoonlijk access token. Het token wordt via getpass ingevoerd zodat het niet zichtbaar is in het notebook. Dit verhoogt de veiligheid, vooral wanneer het notebook gedeeld of opgeslagen wordt in de cloud of in een publieke repository. Zonder deze stap kan het laden van beschermde datasets of modellen mislukken.

In [ ]:
# Login zonder token zichtbaar te maken in je notebook
token = getpass.getpass("Voer je Hugging Face access token in: ")
login(token=token)

## Laden en splitsen van de H&M Fashion Captions dataset

We laden de dataset 'tomytjandra/h-and-m-fashion-caption-12k' via Hugging Face. Deze dataset bevat meer dan 12.000 afbeeldingen van kledingstukken met bijbehorende Engelstalige productomschrijvingen. Elke afbeelding is gekoppeld aan één tekstuele beschrijving, wat ideaal is voor het trainen van een image captioning-model in een supervised learning setting.

Na het laden splitsen we de dataset in een trainings- en validatieset. Dit doen we met een verhouding van 90% training en 10% validatie, zodat het model geleerd gedrag kan generaliseren naar nieuwe, ongeziene data.

Omdat het trainen van grote datasets in Google Colab kan leiden tot geheugenproblemen, beperken we het aantal trainingsvoorbeelden tot 2000 en het aantal validatievoorbeelden tot 300. Deze reductie maakt het mogelijk om het model efficiënt te trainen en evalueren binnen de beperkingen van de beschikbare hardware, zonder daarbij de representativiteit van de data te verliezen.

In [ ]:
#%pip install tf-keras
#!pip install transformers datasets evaluate torchvision huggingface_hub --quiet
# Laad de H&M captioning dataset
dataset = load_dataset("tomytjandra/h-and-m-fashion-caption-12k")

# Split in train/test
dataset = dataset["train"].train_test_split(test_size=0.1)

train_dataset = dataset["train"].select(range(2000))
val_dataset = dataset["test"].select(range(300))   

C:\Users\srlam\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Image Captioning

## Voorbewerking van afbeeldingsdata

In deze stap definiëren we een transformatiepipeline voor de afbeeldingen zodat ze geschikt zijn als input voor het Vision Transformer model (ViT). Omdat ViT-modellen alleen werken met afbeeldingen van vaste afmetingen, schalen we alle afbeeldingen naar 224 bij 224 pixels.

Daarnaast converteren we elke afbeelding naar een PyTorch-tensor, wat nodig is voor verwerking in het neurale netwerk. Deze stappen zijn verplicht voordat de afbeeldingen als input kunnen worden gebruikt in het encoder-gedeelte van het model. Zonder uniforme grootte en tensorrepresentatie zou het model fouten geven tijdens training of inferentie.

In [ ]:
# ChatGPT, 2025, Prompt 1: "Wat is de invoervereiste qua resolutie en tensorformaat voor ViT-architecturen, en hoe beïnvloeden resizing en normalisatie de performance van visuele encoders? Welke interpolatiemethoden behouden details bij downsampling naar 224x224 pixels?"
# Link: https://chatgpt.com/share/6857fcdf-2fbc-8001-ab19-efdbde61a083
# Transformeer afbeeldingen naar inputgrootte voor model
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

## Laden van het voorgetrainde Vision-Text model

In deze stap laden we een bestaand encoder-decoder model dat ontworpen is voor image captioning: het model vit-gpt2-image-captioning. Dit model combineert een Vision Transformer (ViT) als encoder en een GPT-2 taalmodel als decoder.

De encoder zet de visuele informatie van een afbeelding om in een vectorrepresentatie. Deze representatie wordt vervolgens gebruikt door de decoder om een beschrijvende tekst te genereren. Omdat dit model reeds is voorgetraind op grote datasets met afbeeldings-bijschriftparen, kunnen we het effectief fine-tunen op onze kleinere dataset zonder dat we het model vanaf nul hoeven te trainen. Deze benadering valt onder transfer learning, waarbij eerder verworven kennis van een model wordt toegepast op een nieuwe taak, zoals beschreven door Howard en Ruder (2018).

Daarnaast laden we ook de bijbehorende processor, die de afbeeldingen normaliseert en schaalt voordat ze naar het model gaan, en de tokenizer, die captions omzet naar tokens en omgekeerd.

Bron:
Howard, J., & Ruder, S. (2018). Universal Language Model Fine-tuning for Text Classification. In Proceedings of the 56th Annual Meeting of the Association for Computational Linguistics (pp. 328–339). https://arxiv.org/abs/1801.06146

In [ ]:
# ChatGPT, 2025, Prompt 1: "Hoe werkt de encoder-decoderarchitectuur van VisionEncoderDecoderModel technisch? Welke interne mechanismen gebruiken ViT en GPT-2 voor cross-modal alignment? Wat zijn de implicaties van shared embeddings en layer freezing bij transfer learning in deze configuratie "
# Link: https://chatgpt.com/share/6857fd3c-7318-8001-8e5b-1636a7bd0163
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
processor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

## Preprocessing per datapunt

We definiëren een functie die preprocessing uitvoert op elk individueel datapunt uit de dataset. De afbeelding wordt getransformeerd naar een 224x224 tensor met behulp van de eerder gedefinieerde transformaties.

Daarnaast wordt de bijbehorende caption omgezet naar een lijst van tokens met behulp van de tokenizer. De parameter max_length is ingesteld op 32 tokens. Deze waarde is gekozen op basis van verkennend onderzoek: de meeste captions in mode-datasets zijn tussen de 10 en 20 woorden lang. Door iets extra ruimte te laten, vermijden we dat langere captions worden afgekapt, terwijl we toch efficiënt omgaan met padding en rekentijd. Dit sluit aan bij de gemiddelde captionlengtes die zijn waargenomen in het COCO-datasetonderzoek (Lin et al., 2014).

De gegenereerde labels bestaan uit integers die overeenkomen met de tokens in de caption. Deze worden tijdens het trainen vergeleken met de voorspellingen van het model.

Bron:  
Lin, T.-Y., Maire, M., Belongie, S., Hays, J., Perona, P., Ramanan, D., ... & Dollár, P. (2014). Microsoft COCO: Common Objects in Context. *European Conference on Computer Vision (ECCV)*. https://arxiv.org/abs/1405.0312



## Preprocessing toepassen en dataset formatteren

Met de .map()-functie passen we de preprocessing-functie toe op elk datapunt in zowel de trainings- als validatieset. Deze methode zorgt ervoor dat de afbeeldingen en captions automatisch worden omgezet naar het juiste formaat voor modelinput.

Daarna gebruiken we .set_format() om expliciet aan te geven dat we PyTorch-tensors willen gebruiken en welke kolommen daarin nodig zijn. Door de kolommen te beperken tot 'pixel_values' en 'labels' vermijden we onnodige geheugenbelasting, wat cruciaal is bij training op beperkte hardware zoals Google Colab. Deze stap is verplicht om de gegevens correct te kunnen gebruiken in de PyTorch DataLoader in de volgende fase.

In [ ]:
# ChatGPT, 2025, Prompt 1: "Hoe tokeniseer ik captiontekst op een manier die consistent is met de decoder van het model? Hoe verhoudt de keuze van max_length (32 tokens) zich tot de gemiddelde informatiedichtheid van productomschrijvingen in natuurlijke taal? Hoe vertaalt dit zich naar sequentielengte versus padding overhead?"
# ChatGPT, 2025, Prompt 2: "Wat is het voordeel van .map() bij Hugging Face datasets boven for-loops? Hoe behoud ik efficiëntie bij gelijktijdige beeld- en tekstverwerking, en waarom moet ik de dataset herformateren naar een PyTorch-compatibele structuur met gespecificeerde kolommen (zoals pixel_values, labels)?"
# Link: https://chatgpt.com/share/6857fdd3-5518-8001-b6bd-ed3fd9c0cd75

max_length = 32

def preprocess(example):
    image = image_transform(example["image"])
    caption = example["text"]
    labels = tokenizer(caption, padding="max_length", truncation=True, max_length=max_length).input_ids
    return {"pixel_values": image, "labels": labels}

train_dataset = train_dataset.map(preprocess)
val_dataset = val_dataset.map(preprocess)

train_dataset.set_format(type="torch", columns=["pixel_values", "labels"])
val_dataset.set_format(type="torch", columns=["pixel_values", "labels"])

## Train en validatie DataLoaders

We gebruiken PyTorch's DataLoader om de voorbewerkte datasets in batches aan het model aan te bieden. Een batchgrootte van 8 is gekozen als een praktische balans tussen rekensnelheid en het beschikbare geheugen in Google Colab. Een te grote batchgrootte kan leiden tot geheugenfouten, terwijl een te kleine batch de training aanzienlijk vertraagt.

De trainingsset wordt geshuffeld om te voorkomen dat het model leert op basis van de volgorde van de data. Door shuffling krijgt het model steeds andere combinaties van voorbeelden te zien, wat overfitting tegengaat en de generalisatie verbetert.

Voor de validatieset wordt geen shuffling toegepast, zodat de evaluatie consistent en reproduceerbaar blijft.

In [ ]:
# ChatGPT, 2025, Prompt 1: "Wat zijn de geheugeneffecten van het kiezen van een bepaalde batch size in een transformer-gebaseerde captioning pipeline?"
# Link: https://chatgpt.com/share/6857fe34-c27c-8001-8e16-29734fede4c4

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)

## Instellen van optimizer en device

In deze stap kiezen we de optimizer en bepalen we op welk apparaat het model wordt getraind. We gebruiken de AdamW-optimizer, een variant van Adam met decoupled weight decay. Deze optimizer wordt vaak toegepast bij het trainen van transformer-gebaseerde modellen omdat het betere generalisatieprestaties oplevert dan standaard Adam. De weight decay voorkomt overfitting door gewichten af te straffen die te groot worden.

De learning rate is ingesteld op 5e-5, wat een gangbare waarde is voor het fine-tunen van grote voorgetrainde modellen. Een lagere learning rate voorkomt dat het model abrupt zijn reeds aangeleerde kennis overschrijft, wat belangrijk is bij transfer learning.

Het model wordt naar een GPU gestuurd als deze beschikbaar is. Het gebruik van een GPU zorgt voor veel snellere matrixvermenigvuldigingen en trainingsprocessen dan een CPU, vooral bij grote modellen en datasets.

Bron:  
Loshchilov, I., & Hutter, F. (2019). Decoupled Weight Decay Regularization. *International Conference on Learning Representations (ICLR)*. https://arxiv.org/abs/1711.05101

In [ ]:
# ChatGPT, 2025, Prompt 1:Wat is het effect van weight decay in AdamW op het minimaliseren van loss zonder overfitting? Hoe verschilt decoupled weight decay mathematisch van traditionele L2-regularisatie? Waarom is een learning rate van 5e-5 stabiel bij fine-tuning van modellen met LayerNorm en Attention?
# Link: https://chatgpt.com/share/6857fe7c-9edc-8001-92c7-66a581483367

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

## Model trainen en evalueren met BLEU-score

In deze stap trainen we het image captioning-model in meerdere rondes (epochs) en evalueren we het model na elke epoch met behulp van de BLEU-score.

Tijdens training voorspelt het model een reeks woorden (tokens) op basis van de afbeelding. De fout tussen voorspelling en de echte caption wordt gemeten met de CrossEntropyLoss. Deze lossfunctie vergelijkt de gegenereerde tokens met de echte tokens op elke tijdstap.

De formule voor Cross-Entropy Loss is:

![CrossEntropyLoss](https://latex.codecogs.com/png.image?\dpi{150}&space;L&space;=&space;-&space;\sum_{i=1}^{N}&space;y_i&space;\cdot&space;\log(p_i))

waarbij:
- yᵢ = correcte waarde voor token i (one-hot encoded),
- pᵢ = voorspelde kans voor token i.

Een lagere loss betekent dat het model dichter bij de juiste output zit.

### Evaluatie met BLEU

Na elke epoch evalueren we de gegenereerde captions met de BLEU-score. BLEU (Bilingual Evaluation Understudy) vergelijkt n-grammen (reeksen van 1 t/m 4 woorden) van de gegenereerde caption met die van de referentiecaption.

De BLEU-score wordt als volgt berekend:

![BLEU-score](https://latex.codecogs.com/png.image?\dpi{150}&space;BLEU&space;=&space;BP&space;\cdot&space;\exp\left(\sum_{n=1}^N&space;w_n&space;\cdot&space;\log(p_n)\right))

waarbij:
- pₙ = de n-gram precisie voor n=1..4,
- wₙ = het gewicht van elke n-gram (vaak 1/4),
- BP = brevity penalty.

De brevity penalty wordt toegepast om te korte zinnen te bestraffen:

![BrevityPenalty](https://latex.codecogs.com/png.image?\dpi{150}&space;BP&space;=&space;\begin{cases}1&space;&,&space;c&space;>&space;r\\\exp\left(1&space;-&space;\frac{r}{c}\right)&space;&,&space;c&space;\leq&space;r\end{cases})

waarbij:
- c = lengte van de gegenereerde caption,
- r = lengte van de referentiecaption.

Een BLEU-score van 1.0 betekent een perfecte match. In de praktijk zijn scores tussen 0.2 en 0.5 gebruikelijk voor captioningmodellen. Door BLEU te gebruiken kunnen we objectief meten hoe goed het model inhoudelijk en taalkundig overeenkomt met de referentiecaption.

In [ ]:
# ChatGPT, 2025, Prompt 1: "Hoe werkt de generate() methode bij VisionEncoderDecoderModel intern (greedy search, beam search)? Wat is de rol van cross entropy loss bij decoder training, en hoe worden logits gebruikt bij token sampling? Hoe wordt BLEU score gedefinieerd in termen van n-gram precisie, en welke gewichten gelden voor unigram t/m 4-gram evaluatie?"
# Link: https://chatgpt.com/share/6857fed7-60f8-8001-82e6-bb6fc6c10c8b
bleu = evaluate.load("bleu")
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Training epoch {epoch+1}"):
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(pixel_values=pixel_values, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} loss: {total_loss / len(train_loader):.4f}")

    # Validatie met BLEU 
    model.eval()
    preds, refs = [], []
    for batch in tqdm(val_loader, desc=f"Validatie epoch {epoch+1}"):
        pixel_values = batch["pixel_values"].to(device)
        with torch.no_grad():
            generated_ids = model.generate(pixel_values, max_length=max_length)
        decoded_preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(batch["labels"], skip_special_tokens=True)

        preds.extend(decoded_preds)
        refs.extend([[ref] for ref in decoded_labels])

    bleu_score = bleu.compute(predictions=preds, references=refs)
    print(f"BLEU score (epoch {epoch+1}): {bleu_score['bleu']:.4f}")

## Caption genereren op basis van nieuwe afbeelding

Deze functie laat het model een caption genereren op basis van een nieuwe afbeelding. De afbeelding wordt eerst op dezelfde manier getransformeerd als tijdens de training, waarna het model de tokens genereert met behulp van beam search of greedy decoding.

Dit simuleert hoe het model in de praktijk gebruikt zou worden: een gebruiker voert een afbeelding in en ontvangt automatisch een tekstuele beschrijving. De caption wordt gegenereerd door de decoder op basis van de beeldrepresentatie van de encoder.

Deze stap toont aan of het model daadwerkelijk in staat is om betekenisvolle zinnen te vormen die visueel relevante informatie bevatten.

## Vergelijking tussen gegenereerde en echte caption

In deze stap selecteren we willekeurig een voorbeeld uit de validatieset en tonen we de afbeelding samen met de door het model gegenereerde caption en de originele referentie-caption.

Door deze visuele vergelijking te maken krijgen we niet alleen inzicht in de BLEU-score of loss, maar zien we ook hoe goed het model inhoudelijk de afbeelding heeft geïnterpreteerd. Dit is belangrijk voor menselijke evaluatie: een caption kan grammaticaal correct zijn, maar inhoudelijk niet overeenkomen met wat er op de afbeelding staat.

Deze vergelijking ondersteunt dus de kwalitatieve beoordeling van het model en is essentieel bij toepassingen zoals e-commerce, waar correcte beschrijvingen van kleding belangrijk zijn voor gebruikerservaring.


In [ ]:
# ChatGPT, 2025, Prompt 1: "Hoe wordt een afbeelding op inference-structuurniveau verwerkt door ViT naar tekst door GPT-2 binnen een encoder-decoder-architectuur? "
# ChatGPT, 2025, Prompt 2: "Hoe valideer ik de generalisatiecapaciteit van mijn model visueel en semantisch? Wat is het belang van directe vergelijking tussen modeloutput en referentiecaption, en hoe ondersteun ik die beoordeling kwalitatief naast metrische scores?"
# Link: https://chatgpt.com/share/6857ff63-c0b8-8001-8825-938ed388da3f

def generate_caption(image_pil):
    image_tensor = image_transform(image_pil).unsqueeze(0).to(device)
    with torch.no_grad():
        generated_ids = model.generate(image_tensor, max_length=max_length)
    caption = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return caption

# Reset tijdelijk format om toegang te krijgen tot 'text'
val_dataset.reset_format()

# Random index
random_index = random.randint(0, len(val_dataset) - 1)
example = val_dataset[random_index]

true_caption = example["text"]

# Zet list -> tensor -> PIL
img_tensor = torch.tensor(example["pixel_values"])
img_pil = transforms.ToPILImage()(img_tensor)

# Genereer caption
generated_caption = generate_caption(img_pil)

# Toon afbeelding + resultaten
plt.imshow(img_pil)
plt.axis("off")
plt.title("Caption vergelijking", fontsize=14)
plt.show()

print(f"Model-caption    : {generated_caption}")
print(f"Originele caption: {true_caption}")

# Zet format terug voor training
val_dataset.set_format(type="torch", columns=["pixel_values", "labels"])

## Samenvatting van gekozen instellingen en onderbouwing

| Hyperparameter / Instelling     | Gekozen waarde     | Onderbouwing                                                                                                   |
|----------------------------------|---------------------|------------------------------------------------------------------------------------------------------------------|
| **max_length**                   | 32                  | Captions in de dataset zijn gemiddeld 10–20 tokens lang. 32 biedt ruimte zonder overmatig padden. Gebaseerd op analyses van COCO-captionlengtes. |
| **batch_size**                  | 8                   | Te grote batches veroorzaken geheugenproblemen in Colab. 8 biedt balans tussen performance en stabiliteit.     |
| **learning_rate**               | 5e-5                | Kleine waarde voorkomt dat het voorgetrainde model zijn kennis verliest (catastrofaal vergeten). Aanbevolen voor transformer fine-tuning. |
| **num_epochs**                  | 3                   | Voldoende om op kleine subset convergence te bereiken zonder overfitting. Training is stabiel en efficiënt.     |
| **optimizer**                   | AdamW               | Werkt beter dan Adam in combinatie met weight decay (betere generalisatie). Gebaseerd op literatuur (Loshchilov & Hutter, 2019). |
| **model**                       | ViT-GPT2 (nlpconnect/vit-gpt2-image-captioning) | Pre-trained encoder-decoder model specifiek ontwikkeld voor image captioning. Werkt direct met beelden en genereert natuurlijke taal. |
| **dataset**                     | H&M Captions (Hugging Face) | Dataset bevat realistische productomschrijvingen voor mode. Perfect voor fine-tuning op beeld-bijschrift taken. |
| **eval-metriek**                | BLEU (1–4 gram)     | Standaardmaat voor automatische tekstgeneratie. Meet overlap tussen gegenereerde tekst en referentiecaption.   |
| **datasplitsing**              | 2000 training / 300 validatie | Om geheugenverbruik in Colab te beperken én het model toch effectief te trainen en valideren.                 |

# Beeldgeneratie

In [ ]:
# Kies een voorgetraind BLIP model voor image captioning
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Voorbeeld preprocessing functie
def preprocess(example):
    inputs = processor(images=example["image"], text=example["text"], return_tensors="pt", padding="max_length", truncation=True)
    inputs = {k: v.squeeze() for k, v in inputs.items()}
    inputs["labels"] = inputs["input_ids"]
    return inputs

# Gebruik with_transform voor memory-efficiënte preprocessing
train_tokenized_dataset = train_dataset.with_transform(preprocess)
val_tokenized_dataset = val_dataset.with_transform(preprocess)

# Stel training parameters in
training_args = TrainingArguments(
    output_dir="./blip-finetuned-hm",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    save_steps=500,
    logging_steps=100,
    learning_rate=5e-5,
    fp16=True,
    report_to="none",
    remove_unused_columns=False 
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_dataset,
    eval_dataset=val_tokenized_dataset,
    tokenizer=processor,
)

# Fine-tune het model
trainer.train()

C:\Users\srlam\AppData\Local\Temp\ipykernel_5824\3926884792.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
C:\Users\srlam\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


## Modelarchitectuur en Pretraining

Voor deze taak is gekozen voor de **BLIP (Bootstrapped Language-Image Pretraining)** modelarchitectuur, specifiek het model `Salesforce/blip-image-captioning-base`. Dit model is vooraf getraind op grote multimodale datasets zoals **Conceptual Captions** en **LAION-400M**, waarbij het leert om afbeeldingen en tekstuele beschrijvingen aan elkaar te koppelen.

## Taak: Tekst-naar-beeld generatie

Het model wordt aangepast zodat het een **Engels tekstbijschrift als input** neemt en een **afbeelding als output** genereert. In deze notebook wordt BLIP gebruikt voor image captioning, maar voor tekst-naar-beeld generatie zou een model als **Stable Diffusion** of **DALL·E** meer geschikt zijn. BLIP kan echter ook multimodale taken uitvoeren, zoals het genereren van afbeeldingen op basis van tekst (mits aangepast).

## Stappen bij het finetunen

1. **Dataset laden:** De H&M fashion captioning dataset wordt geladen en gesplitst in train/validatie.
2. **Preprocessing:** Elke sample wordt verwerkt met de BLIP processor, waarbij de tekst en afbeelding worden omgezet naar tensors die het model kan gebruiken.
3. **Tokenisatie:** De tekst wordt getokeniseerd en samen met de afbeelding als input aan het model gegeven.
4. **Training setup:** Trainingsparameters worden ingesteld, zoals batch size, aantal epochs, learning rate, enz.
5. **Trainer:** De Huggingface `Trainer` wordt gebruikt om het model te finetunen op de trainingsdata.
6. **Validatie:** Tijdens training wordt het model geëvalueerd op de validatieset.

## Componenten van het model

- **Vision Encoder:** Een visueel neuraal netwerk (ViT) dat afbeeldingsfeatures extraheert.
- **Text Encoder:** Een transformer die tekstuele input verwerkt.
- **Cross-modal Transformer:** Combineert visuele en tekstuele informatie.
- **Decoder:** Genereert de output (bijvoorbeeld een caption of, bij uitbreiding, een afbeelding).

## Hoe maakt het model afbeeldingen en welke lossfunctie gebruikt het?

Het BLIP-model leert een koppeling tussen tekst en beeld. Bij image captioning genereert het tekst bij een afbeelding; bij tekst-naar-beeld (mits aangepast) zou het een afbeelding genereren die past bij de tekst. De **lossfunctie** is meestal een combinatie van cross-entropy loss voor tekstgeneratie en contrastieve loss voor multimodale matching.

## Keuze van hyperparameters, optimizer en epochs

- **Batch size:** 8, gekozen voor een balans tussen snelheid en geheugengebruik.
- **Epochs:** 3, voldoende om te finetunen zonder overfitting op een relatief kleine dataset.
- **Learning rate:** 5e-5, standaardwaarde voor finetuning van transformer-gebaseerde modellen.
- **Optimizer:** AdamW, omdat deze goed werkt voor grote taal- en visiemodellen door adaptieve learning rates en gewichtsregularisatie.

Deze keuzes zijn gebaseerd op best practices voor het finetunen van grote multimodale modellen en de grootte van de dataset.

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def fashion_app(image_path, detection_model, caption_model, tokenizer, image_transform, class_names, device, max_length=32):
    # 1. Detectie: Voorspel bounding boxes en klassen
    results = detection_model(image_path)
    boxes = results[0].boxes.xyxy.cpu().numpy()  # [x1, y1, x2, y2]
    class_ids = results[0].boxes.cls.cpu().numpy().astype(int)
    scores = results[0].boxes.conf.cpu().numpy()
    
    # 2. Laad afbeelding
    image = Image.open(image_path).convert("RGB")
    fig, ax = plt.subplots(1, figsize=(10, 10))
    ax.imshow(image)
    
    # 3. Voor elk gedetecteerd object: crop, caption, plot
    captions = []
    for i, (box, class_id, score) in enumerate(zip(boxes, class_ids, scores)):
        x1, y1, x2, y2 = map(int, box)
        cropped = image.crop((x1, y1, x2, y2))
        # Caption genereren
        image_tensor = image_transform(cropped).unsqueeze(0).to(device)
        with torch.no_grad():
            generated_ids = caption_model.generate(image_tensor, max_length=max_length)
        caption = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        captions.append((class_names[class_id], caption))
        # Plot bounding box en caption
        rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=2, edgecolor='lime', facecolor='none')
        ax.add_patch(rect)
        ax.text(x1, y1-10, f"{class_names[class_id]}: {caption}", color='white', fontsize=10,
                bbox=dict(facecolor='black', alpha=0.7, edgecolor='none'))
    ax.axis('off')
    plt.show()
    return captions

# --- Gebruik de app op een voorbeeldafbeelding ---
# Pas het pad hieronder aan naar jouw testafbeelding
test_image_path = "voorbeeld.jpg"  # <-- Vervang door jouw afbeelding

# Voer de app uit
resultaten = fashion_app(
    image_path=test_image_path,
    detection_model=model,           # YOLOv8 model
    caption_model=model,             # Captioning model (ViT-GPT2)
    tokenizer=tokenizer,
    image_transform=image_transform,
    class_names=CLASS_NAMES,
    device=device,
    max_length=32
)

# Print resultaten
for cls, cap in resultaten:
    print(f"{cls}: {cap}")